In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.818490,False,17,1,42.0
1,8.141671,False,29,1,38.0
2,5.948072,True,38,1,38.0
3,8.838332,True,27,1,39.0
4,9.259415,True,28,1,38.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9991.000000,10000.000000,10000.000000,9881.000000
mean,7.279259,27.332300,1.033200,38.704888
std,1.317603,6.238122,0.188948,2.602659
min,0.661387,12.000000,1.000000,19.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.808857,53.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5158
False    4842
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [8]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [9]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [10]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6665 samples, validate on 741 samples
Epoch 1/10
6665/6665 [==============================] - 2s 320us/sample - loss: 2.4300 - mae: 1.0898 - mse: 2.4300 - val_loss: 1.9330 - val_mae: 1.0964 - val_mse: 1.9330
Epoch 2/10
6665/6665 [==============================] - 1s 132us/sample - loss: 1.3620 - mae: 0.9110 - mse: 1.3620 - val_loss: 2.6606 - val_mae: 1.3270 - val_mse: 2.6606
Epoch 3/10
6665/6665 [==============================] - 1s 176us/sample - loss: 1.3475 - mae: 0.9075 - mse: 1.3475 - val_loss: 1.2050 - val_mae: 0.8572 - val_mse: 1.2050
Epoch 4/10
6665/6665 [==============================] - 1s 136us/sample - loss: 1.3105 - mae: 0.8972 - mse: 1.3105 - val_loss: 1.2329 - val_mae: 0.8563 - val_mse: 1.2329
Epoch 5/10
6665/6665 [==============================] - 1s 190us/sample - loss: 1.2971 - mae: 0.8908 - mse: 1.2971 - val_loss: 1.2201 - val_mae: 0.8585 - val_mse: 1.2201
Epoch 6/10
6665/6665 [==============================] - 1s 138us/sample - loss: 1.2806 - mae: 0.8854 - 

In [11]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [12]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  6.166089
Actual val:  5.43659938092

Predicted val:  7.631364
Actual val:  7.62578964258

Predicted val:  7.548973
Actual val:  6.0847584312

Predicted val:  7.917487
Actual val:  7.0327461578

Predicted val:  7.49281
Actual val:  6.3382900325

Predicted val:  8.233382
Actual val:  8.99926953484

Predicted val:  7.1377306
Actual val:  5.5005334369

Predicted val:  7.5802665
Actual val:  6.3118345610599995

Predicted val:  5.3358426
Actual val:  2.20462262

Predicted val:  7.898989
Actual val:  8.62448368944



In [15]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [16]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [17]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…